In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [ ]:
# 하이퍼파라미터 설정
learning_rate = 0.001
num_epochs = 50
batch_size = 64

In [ ]:
# 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print("Device:", device)

Device: cuda


In [ ]:
#GPT가 성능을 높이는 방법으로 알려준 코드
# 데이터 전처리 및 증강을 위한 transforms 설정 (훈련용)
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),   # 이미지 좌우 랜덤 반전
    transforms.RandomCrop(28, padding=4), # 랜덤하게 이미지를 자르고 패딩 추가
    transforms.ToTensor()   # 이미지를 텐서로 변환
])

transform_test = transforms.ToTensor()

# 데이터셋: FashionMNIST 로드
train_dataset = dsets.FashionMNIST(root='FashionMNIST_data/', train=True,
                                    transform=transform_train, download=True)
test_dataset = dsets.FashionMNIST(root='FashionMNIST_data/', train=False,
                                   transform=transform_test, download=True)
# 데이터 로더 (데이터셋을 배치 단위로 로드)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

NameError: name 'transforms' is not defined

In [ ]:
# 데이터 전처리 및 증강을 위한 transforms 설정 (훈련용)# CNN 모델 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
         # 첫 번째 합성곱 레이어
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # 두 번째 합성곱 레이어
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # 세 번째 합성곱 레이어
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # 드롭아웃으로 과적합 방지
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(3 * 3 * 128, 625)
        self.fc2 = nn.Linear(625, 10)

        # 가중치 초기화
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)  # flatten
        out = self.dropout(out)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
# 모델, 손실함수, 옵티마이저 정의
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 모델 학습
for epoch in range(num_epochs):
    avg_loss = 0
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()   # 기울기 초기화
        loss.backward()   # 기울기 계산
        optimizer.step()  # 모델 파라미터 업데이트

        avg_loss += loss / len(train_loader)

    print(f'[Epoch {epoch + 1}] Loss: {avg_loss:.4f}')

[Epoch 1] Loss: 0.6427
[Epoch 2] Loss: 0.4259
[Epoch 3] Loss: 0.3769
[Epoch 4] Loss: 0.3447
[Epoch 5] Loss: 0.3254
[Epoch 6] Loss: 0.3100
[Epoch 7] Loss: 0.2986
[Epoch 8] Loss: 0.2887
[Epoch 9] Loss: 0.2812
[Epoch 10] Loss: 0.2736
[Epoch 11] Loss: 0.2662
[Epoch 12] Loss: 0.2615
[Epoch 13] Loss: 0.2544
[Epoch 14] Loss: 0.2505
[Epoch 15] Loss: 0.2450
[Epoch 16] Loss: 0.2399
[Epoch 17] Loss: 0.2379
[Epoch 18] Loss: 0.2354
[Epoch 19] Loss: 0.2340
[Epoch 20] Loss: 0.2314
[Epoch 21] Loss: 0.2244
[Epoch 22] Loss: 0.2251
[Epoch 23] Loss: 0.2185
[Epoch 24] Loss: 0.2203
[Epoch 25] Loss: 0.2158
[Epoch 26] Loss: 0.2117
[Epoch 27] Loss: 0.2133
[Epoch 28] Loss: 0.2077
[Epoch 29] Loss: 0.2074
[Epoch 30] Loss: 0.2080
[Epoch 31] Loss: 0.2037
[Epoch 32] Loss: 0.2008
[Epoch 33] Loss: 0.2022
[Epoch 34] Loss: 0.2010
[Epoch 35] Loss: 0.1969
[Epoch 36] Loss: 0.1972
[Epoch 37] Loss: 0.1963
[Epoch 38] Loss: 0.1937
[Epoch 39] Loss: 0.1953
[Epoch 40] Loss: 0.1897
[Epoch 41] Loss: 0.1895
[Epoch 42] Loss: 0.1871
[

In [ ]:
# 정확도 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy:.2f}%')

Accuracy on test set: 92.91%
